In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pyxdf
import soundcard as sc
import wave
import struct
import re
import csv
import pprint
import json
from mdutils.mdutils import MdUtils

# PYXDF only works with base vm 

In [4]:
def find_xdf_files(folder_path):
    xcf_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.xdf'):
                xcf_files.append(os.path.join(root, file))
    return xcf_files

In [228]:
folder_path = "/home/nishal/PROJECTS/MUSMET/sae_barcelona/Recordings_1104_SAE_BCN-20250414T143655Z-002" # Replace with your folder path
working_files = []
working_idx = []

xdf_files = find_xdf_files(folder_path)
for i in range(len(xdf_files)):
    if(xdf_files[i][-7:]=='eeg.xdf'):
        working_files.append(xdf_files[i])
        print(i, xdf_files[i])
        working_idx.append(i)


0 /home/nishal/PROJECTS/MUSMET/sae_barcelona/Recordings_1104_SAE_BCN-20250414T143655Z-002/Recordings_1104_SAE_BCN/piano_happy/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf
1 /home/nishal/PROJECTS/MUSMET/sae_barcelona/Recordings_1104_SAE_BCN-20250414T143655Z-002/Recordings_1104_SAE_BCN/guitar_relax/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf
2 /home/nishal/PROJECTS/MUSMET/sae_barcelona/Recordings_1104_SAE_BCN-20250414T143655Z-002/Recordings_1104_SAE_BCN/PIANO_AGGR/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf
3 /home/nishal/PROJECTS/MUSMET/sae_barcelona/Recordings_1104_SAE_BCN-20250414T143655Z-002/Recordings_1104_SAE_BCN/Guitar_aggr/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf
4 /home/nishal/PROJECTS/MUSMET/sae_barcelona/Recordings_1104_SAE_BCN-20250414T143655Z-002/Recordings_1104_SAE_BCN/PIANO_RELAX/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf


In [229]:
# print(xdf_files[file_path].split('/')[8])
print(working_idx)
# wiorking_idx = [2]

for idx in working_idx:
    file_path = idx
    foldername = 'exports/' + xdf_files[file_path].split('/')[8]
    print(foldername)

    if not os.path.exists(foldername):
        os.makedirs(foldername)

    data, header = pyxdf.load_xdf(xdf_files[file_path])

    mdFile = MdUtils(file_name='exports/' + xdf_files[file_path].split('/')[8]+'/README', title=xdf_files[file_path].split('/')[8])
    mdFile.new_header(level=1, title='Streams Recorded')

    table_data = ['File Name', 'Stream Type', 'Stream Length', 'hostname']
    for i, stream in enumerate(data):
        if stream['info']['name'][0] == 'AudioStream':
            table_data.append(stream['info']['hostname'][0] + '_' + stream['info']['name'][0] + '.wav')
        else:
            table_data.append(stream['info']['hostname'][0] + '_' + stream['info']['name'][0] + '.csv')

        table_data.append(stream['info']['type'][0])
        table_data.append(str(len(stream['time_series'])))
        table_data.append(stream['info']['hostname'][0])

    mdFile.new_table(columns=4, rows=int(len(table_data)/4), text=table_data)

    mdFile.create_md_file()


    for i, stream in enumerate(data):
        filename = foldername + '/' + stream['info']['hostname'][0] + '_' + stream['info']['name'][0]
        if stream['info']['name'][0] == 'AudioStream':
            save_wav_file(stream, filename)
        else:
            save_csv_file(stream, filename)


        # if stream['info']['name'][0] != 'AudioStream':
        #     filename = foldername + '/' + stream['info']['hostname'][0] + '_' + stream['info']['name'][0]
        #     print(filename)
        #     save_timestamps(stream, filename)
        #     save_data(stream, filename)


[0, 1, 2, 3, 4]
exports/piano_happy
	 Saving exports/piano_happy/Coworking3_UnicornRecorderRawDataLSLStream.csv
	 Saving timestamps exports/piano_happy/Coworking3_UnicornRecorderRawDataLSLStream.csv
	 Saving exports/piano_happy/Coworking3_OSC2LSL.csv
	 Saving timestamps exports/piano_happy/Coworking3_OSC2LSL.csv
	 Saving exports/piano_happy/Coworking3_UnicornRecorderLSLStream.csv
	 Saving timestamps exports/piano_happy/Coworking3_UnicornRecorderLSLStream.csv
	 Saving exports/piano_happy/Coworking2_UnicornRecorderRawDataLSLStream.csv
	 Saving timestamps exports/piano_happy/Coworking2_UnicornRecorderRawDataLSLStream.csv
	 Saving exports/piano_happy/Coworking2_OSC2LSL.csv
	 Saving timestamps exports/piano_happy/Coworking2_OSC2LSL.csv
	 Saving exports/piano_happy/Coworking2_UnicornRecorderLSLStream.csv
	 Saving timestamps exports/piano_happy/Coworking2_UnicornRecorderLSLStream.csv
	 Saving exports/piano_happy/Coworking3_AudioStream.wav
	 Saving exports/piano_happy/Coworking2_AudioStream.wa

In [196]:
file_path = 4
print(xdf_files[file_path].split('/')[8])

foldername = 'exports/' + xdf_files[file_path].split('/')[8]
if not os.path.exists(foldername):
    os.makedirs(foldername)

data, header = pyxdf.load_xdf(xdf_files[file_path])
for i, stream in enumerate(data):
    print(f"Stream {i}: {len(stream['time_series'])} |   Name: {stream['info']['name'][0]}")


mdFile = MdUtils(file_name='exports/' + xdf_files[file_path].split('/')[8]+'/README', title=xdf_files[file_path].split('/')[8])
mdFile.new_header(level=1, title='Streams Recorded')

table_data = ['File Name', 'Stream Type', 'Stream Length', 'hostname']
for i, stream in enumerate(data):
    if stream['info']['name'][0] == 'AudioStream':
        table_data.append(stream['info']['hostname'][0] + '_' + stream['info']['name'][0] + '.wav')
    else:
        table_data.append(stream['info']['hostname'][0] + '_' + stream['info']['name'][0] + '.csv')

    table_data.append(stream['info']['type'][0])
    table_data.append(str(len(stream['time_series'])))
    table_data.append(stream['info']['hostname'][0])

mdFile.new_table(columns=4, rows=int(len(table_data)/4), text=table_data)

mdFile.create_md_file()



for i, stream in enumerate(data):
    filename = foldername + '/' + stream['info']['hostname'][0] + '_' + stream['info']['name'][0]
    if stream['info']['name'][0] == 'AudioStream':
        save_wav_file(stream, filename)
    else:
        save_csv_file(stream, filename)

PIANO_RELAX
Stream 0: 17578 |   Name: OSC2LSL
Stream 1: 60144 |   Name: UnicornRecorderRawDataLSLStream
Stream 2: 60142 |   Name: UnicornRecorderLSLStream
Stream 3: 60111 |   Name: UnicornRecorderLSLStream
Stream 4: 60111 |   Name: UnicornRecorderRawDataLSLStream
Stream 5: 17492 |   Name: OSC2LSL
Stream 6: 11531264 |   Name: AudioStream
Stream 7: 11525120 |   Name: AudioStream


In [ ]:
# for i, stream in enumerate(data):
#     filename = foldername + '/' + stream['info']['hostname'][0] + '_' + stream['info']['name'][0]
#     if stream['info']['name'][0] == 'AudioStream':
#         save_wav_file(stream, filename)
#     else:
#         save_csv_file(stream, filename)

In [144]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [220]:
def save_wav_file(stream, filename):
    print(f"\t Saving {filename}.wav")

    scaled_data = (stream['time_series'] * 32767).astype(np.int16)

    sample_rate = 48000
    num_channels = 2
    sampwidth = 2  # Bytes per sample (16-bit PCM)

    with wave.open(filename+'.wav', "w") as wav_file:
        wav_file.setnchannels(num_channels)
        wav_file.setsampwidth(sampwidth)
        wav_file.setframerate(sample_rate)
        
        # Pack and write frames
        frames = b''.join([struct.pack('hh', left, left) for left, right in scaled_data])
        wav_file.writeframes(frames)


    with open(filename+'.json', 'w') as f:
        json.dump(stream['info'], f, indent=2, cls=NpEncoder)        

In [226]:
def save_csv_file(stream, filename):
    print(f"\t Saving {filename}.csv")
    
    np.savetxt(filename+'.csv', stream['time_series'], delimiter=',', fmt='%s')

    with open(filename+'.json', 'w') as f:
        json.dump(stream['info'], f, indent=2, cls=NpEncoder)

    print(f"\t Saving timestamps {filename}.csv")
    np.savetxt(filename+'_timestamps.csv', stream['time_stamps'], delimiter=',', fmt='%s')


In [225]:
# print(stream['time_stamps'])

# # write timestamps to csv
# def save_timestamps(stream, filename):
#     np.savetxt(filename+'_timestamps.csv', stream['time_stamps'], delimiter=',', fmt='%s')

In [215]:
# mdFile = MdUtils(file_name='Example_Markdown', title='Markdown File Example')
# mdFile.new_header(level=1, title='Section 1')
# mdFile.new_paragraph('This is a paragraph.')
# mdFile.new_list(['Item 1', 'Item 2', 'Item 3'])
# mdFile.create_md_file()


In [214]:
# # pprint.pprint(data[1]['info'])
# # pprint.pprint(data[7]['info'])

# for key, value in data[2]['info'].items():
#     if isinstance(value, list) and len(value) == 1:
#         print(key, value[0])
#     elif isinstance(value, list) and len(value) == 0:
#         print(key, None)
#     else:
#         print(key, value)


#     # print(key, type(value), value)


# # with open('output.json', 'w') as f:
# #     json.dump(data[1]['info'], f, indent=2)
#     # for key, value in data[1]['info'].items():
#     #     if isinstance(value, list) and len(value) == 1:
#     #         json.dump({key: value[0]}, f)
#     #         f.write('\n')
#     #     elif isinstance(value, list) and len(value) == 0:
#     #         json.dump({key: None}, f)
#     #         f.write('\n')
#     #     else:
#     #         json.dump({key: value}, f)
#     #         f.write('\n')


In [213]:
# stream = data[3]
# print(foldername + '/' + stream['info']['name'][0] + '_' + stream['info']['hostname'][0] + '.csv')
# stream = data[7]
# print(foldername + '/' + stream['info']['name'][0] + '_' + stream['info']['hostname'][0] + '.csv')

In [212]:
# stream = data[2]
# filename = foldername + '/' + stream['info']['hostname'][0] + '_' + stream['info']['name'][0] + '.csv'

# np.savetxt(filename, stream['time_series'], delimiter=',', fmt='%s')
# # file_path = 2
# # data, header = pyxdf.load_xdf(xdf_files[file_path])
# # for i, stream in enumerate(data):
# #     print(f"Stream {i}: {len(stream['time_series'])} |   Name: {stream['info']['name'][0]}")


In [211]:
# stream = data[1]

# scaled_data = (stream['time_series'] * 32767).astype(np.int16)

# sample_rate = 48000
# num_channels = 2
# sampwidth = 2  # Bytes per sample (16-bit PCM)

# filename = foldername + '/' + stream['info']['name'][0] + '.wav'

# if os.path.exists(filename):
#     filename = foldername + '/' + stream['info']['name'][0] + '_1.wav'


# with wave.open(filename, "w") as wav_file:
#     wav_file.setnchannels(num_channels)
#     wav_file.setsampwidth(sampwidth)
#     wav_file.setframerate(sample_rate)
    
#     # Pack and write frames
#     frames = b''.join([struct.pack('hh', left, left) for left, right in scaled_data])
#     wav_file.writeframes(frames)


